## Python code for 
### Batch Mode Active Sampling Based on Marginal Probability Distribution Matching

This algorithm selects a set of query instances such that the marginal distribution represented by the selected instances is closest to the distribution represented by the unlabeled data. 

In other words, in order to learn a classifier with a budgeted number of labeled data
this algorithm selects a set of samples $Q$ from the unlabeled set of instances $U$, such that the probability distributions represented by $L \cup Q$ and $U \setminus Q$ are similar to each other, where $L$ is the set of available labeled instances.

##### Reference: 
Chattopadhyay, Rita, et al. "Batch mode active sampling based on marginal probability distribution matching." ACM Transactions on Knowledge Discovery from Data (TKDD) 7.3 (2013): 13.

### Import required libraries

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np

### Load Data

In [ ]:
base = 'data\\'

In [ ]:
df_L = pd.read_csv(base+'L.csv', encoding='utf8')
df_L.head(5)

In [ ]:
df_U = pd.read_csv(base+'U.csv', encoding='utf8')
df_U.head(5)

In [ ]:
np.dot(np.array([[1, 1, 1]]), np.array([[1], [1], [1]]))

## Define required functions

In [ ]:
def get_V(L, U):
    """ Generates matrix V. Each column in V contains v_k; v_k is the vector of instance x_k in (L union U)
    
    inputs: 
        L: a list of labelled instances
        U: a list of unlabelled instances
    returns:
        count_vectorizer: count vectorizer
        V: matrix of v_k vectors of instances (row vector format)
        Vt: transposed matrix of V (column vector format)
    """ 
    count_vectorizer = CountVectorizer()
    V = count_vectorizer.fit_transform(U + L)
    return count_vectorizer, V.toarray()

In [ ]:
count_vectorizer, V = get_V(L=list(df_L['text']), U=list(df_U['text']))
V[0]

### Compute $K_1, k_2$ and $k_3$ as explained in section 2.2.

  1. Kernel gram matrix G is a $(n_u + n_l)$ by $(n_u + n_l)$ matrix
  
  2. Kernel function K such that $G(i, j) = K(x_i, x_j)$
  
  3. $K_1 = G(1 : n_u, 1 : n_u)$
 
  4. $k_2(i) = \frac{n_l + b}{n_l + n_u} \sum_{j=1}^{n_u} K_1(i, j)$
  
  5. $k_3(i) = \frac{n_u - b}{n_l + n_u} \sum_{j=1}^{n_l} G(i, n_u + j)$
  
Where
  1. $X = {x_1, \dots, x_n}$ is a set of instances
  2. $n_u$ is the size of $U$, where $U$ is the set of unlabelled instances
  3. $n_l$ is the size of $L$, where $L$ is the set of unlabelled instances
  4. Kernel gram matrix a set of vectors $v_1, \dots, v_n$ whose entries are given by $G_{i, j} = \left<v_i, v_j \right>$. For finite-dimensional real vectors in $\mathbb{R}^n$ with the usual Euclidean dot product, the Gram matrix $G$ is simply $G=V^{\mathrm {T} }V$, where $V$ is a matrix whose columns are the vectors $v_{k}$. More specifically: for the linear kernel, the Gram matrix is simply the inner product $G_{i,j} = x_{i}^T x_j$. For other kernels, it is the inner product in a feature space with feature map $\phi$: i.e. $G_{i,j} = \phi(x_i)^T \phi(x_j)$
  

In [ ]:
def get_G(V=None):
    """ Calculates the kernel gram matrix G 
    
    inputs: 
        V: matrix of v_k vectors of instances in (L union U)
    returns: 
        G: kernel gram matrix
    """
    G = np.matmul(np.array(V), np.array(V).transpose())
    
    # codes here
    return G

In [ ]:
G = get_G(V=V)
G

In [ ]:
def get_K1(G=None, nu=0):
    """ Extracts entries of unlabelled instances from G
    
    inputs: 
        G: kernel gram matrix
    returns: 
        K1: returns matrix K1
    """
    return G[:nu, 0:nu]

In [ ]:
K1 = get_K1(G=G, nu=len(df_U))
K1

In [ ]:
def get_k2_i(i=0, nl=0, nu=0, b=0, K1=None):
    # returns scaler k2(i)
    return (nl+b)/(nl+nu)*sum(K1[i])

def get_k2(nl=0, nu=0, b=0, K1=None):
    # returns vector k2
    k2 = []
    for i in range(len(K1)):
        k2.append(get_k2_i(i=i, nl=nl, nu=nu, b=b, K1=K1))
    return k2

In [ ]:
b = 10 # set batch size

In [ ]:
k2 = get_k2(nl=len(df_L), nu=len(df_U), b=b, K1=K1)
k2[0]

In [ ]:
def get_k3_i(i=0, nl=0, nu=0, b=0, G=None):
    # returns skeller k3(i)
    return (nu-b)/(nu+nl)*sum(G[i][nu-1:])

def get_k3(K1len=0, nl=0, nu=0, b=0, G=None):
    # returns vector k3
    k3 = []
    for i in range(K1len):
        k3.append(get_k3_i(i=i, nl=nl, nu=nu, b=b, G=G))
    return k3

In [ ]:
k3 = get_k3(K1len=len(K1), nl=len(df_L), nu=len(df_U), b=b, G=G)
k3[0]

In [ ]:
def compute_K1_k2_k3(V=None, nu=0, nl=0):
    G = get_G(V)
    K1 = get_K1(G, len(df_U))
    k2 = get_k2(nl=len(df_L), nu=len(df_U), b=b, K1=K1)
    k3 = get_k3(K1len=len(K1), nl=len(df_L), nu=len(df_U), b=b, G=G)
    
    return K1, k2, k3

In [ ]:
def form_D_matrix(K1=None, k2=None, k3=None):
    # compute D matrix as explained in section 2.3
    D = []
    for i in range(len(K1)):# each row of K1 
        v = []
        for j in range(len(K1[0])):
            if i==j:
                e = K1[i][j] - k2[i] + k3[i]
            else:
                e = K1[i][j]
            v.append(e)
        D.append(v)
            
    return D

In [ ]:
D = form_D_matrix(K1=K1, k2=k2, k3=k3)
D[0][:10]

### compute alpha (i.e. $\alpha$) by solving equation 13
1. $\alpha$ (i.e. alpha)  a binary vector, i.e. each entry in $\alpha$ is either 0 or 1. 
2. If an instance is selected, the corresponding entry in vector $\alpha$ is 1 else 0.
3. The cost function in equation 5 is: $f(Q) = \left | \left | \frac{1}{n_l+b}\sum_{j\in L\cup Q}\Phi(x_j) - \frac{1}{n_u-b} \sum_{i\in U \setminus Q} \Phi(x_i)\right | \right |^2_\mathcal{H}$
4. The minimization problem is to finding the $\alpha$ that minimizes the cost function f(Q) in Equation 5, which can be reformulated as equation 13 as follows:
$\min_{\alpha} \frac{1}{2} \sum_{d_{i,j}<0} d_{i,j}\times (\alpha_i+\alpha_j) + \sum_{d_{i,j}>0} d_{i,j} \times (\alpha_i \times \alpha_j)$

P.N. The above optimisation is writen as follows in the paper:
$\min_{\alpha} \frac{1}{2} \sum_{d_{i,j}<0} d_{i,j}\times (\alpha_i+\alpha_j) + \sum_{d_{i,j}>0} d_{i,j} \times z_{i,j}$, where $z_{i,j} = (\alpha_i \times \alpha_j)$

In [ ]:
def cost_fun(D, alpha):
    cost = 0
    for i in range(len(D)):
        for j in range(len(D[0])):
            if D[i][j]<0:
                cost += D[i][j]*(alpha[i]+alpha[j])*0.5
            else:
                cost += D[i][j]*alpha[i]*alpha[j]
    return cost

def compute_alpha(nu=0, b=0, D=None):
    """ computes binary vector alpha
    
    inputs:
        D: D matrix
        nu: number of unlabelled instances
        b: batch size
    returns: alpha
    """
    
    alpha = []
    alpha = np.array([0] * nu) # initialize alpha to zeros
    # update alpha iteratively for unlabelled instances
    pre_cost = cost_fun(D, alpha)
    for i in range(nu):
        alpha[i] = 1 
        curr_cost = cost_fun(D, alpha)
        if curr_cost >= pre_cost: 
            alpha[i] = 0
    return alpha

In [ ]:
alpha = compute_alpha(nu=len(df_U),  b=b, D=D)
alpha[:20]

In [ ]:
sum(alpha)

In [ ]:
def sort_U_by_alpha(U=None, alpha=alpha):
    keys, U = zip(*sorted(zip(alpha, U)))
    return U

In [ ]:
U = sort_U_by_alpha(list(df_U['text']), alpha=alpha)
U[:5]

In [ ]:
def get_top_b(U=None, b=10):
    return U[:b]

In [ ]:
Q = get_top_b(U=U, b=b)
Q[:5]

In [ ]:
def update_U(U=None, Q=None):
    """ updates L and U
    
    inputs:
        U: list of unlabelled instances
        Q: list of query instances
    returns:
        U: updated U
    """
    
    # code to update U
    U = list(set(U)-set(Q))
    
    return U

In [ ]:
U = update_U(U=U, Q=Q)
U[:5]

In [ ]:
# main cell
# This is the algorithm MP-AL (for LP Problem). 
# Or we can skip calling corresponding functions above and call this function.

def marginal_prob_algo(L=None, U=None, b=10):
    """ Batch Mode Active Sampling Based on Marginal Probability Distribution Matching
    
    inputs:
        L: a set of labelled instances
        U: a set of unlabelled instances
        b: batch size
        
    returns: 
        Q: query set, i.e. a set of instances selected for labelling
    
    """
    count_vectorizer, V = get_V(L=L, U=U)
    G = get_G(V=V)
    K1, k2, k3 = compute_K1_k2_k3(V=V, nu=len(U), nl=len(L))
    D = form_D_matrix(K1=K1, k2=k2, k3=k3)
    alpha = compute_alpha(nu=len(U), b=b, D=D)
    U = sort_U_by_alpha(U=U, alpha=alpha)
    Q = get_top_b(U=U, b=b)
    return U, Q

In [ ]:
# We can skip calling this function because U and Q already collected in previous cells.
# U, Q = marginal_prob_algo(L=list(df_L['text']), U=list(df_U['text']), b=b)

### Store Q for labellig, and store updated U

In [ ]:
# store Q for labellig
# store updated U

df_Uu = pd.DataFrame(zip([0]*len(U), U), columns=['label', 'text'])
df_Q = pd.DataFrame(zip([0]*len(Q), Q), columns=['label', 'text'])

In [ ]:
df_Uu.head()

In [ ]:
df_Uu.to_csv(base+'Uu.csv', index=None, encoding='utf8')

In [ ]:
df_Q.head()

In [ ]:
df_Q.to_csv(base+'Q.csv', index=None, encoding='utf8')

# Please let me know if you have any feedback. 